In [7]:
import sys
import os

PROJECT_ROOT = os.path.abspath("..")
if PROJECT_ROOT not in sys.path:
    sys.path.append(PROJECT_ROOT)


In [8]:
import pandas as pd
from src.preprocessing.text_cleaner import preprocess_dataframe

df = pd.read_csv("../data/raw/tickets.csv")
df = preprocess_dataframe(df)

df.head()


,ticket_id,subject,description,channel,category,priority,created_at,customer_plan,sla_breached,escalated,support_sentiment,combined_text,clean_text
0,1,Unable to login with my account,Unable to login with my account. Kindly check ...,email,Login,High,2026-02-08 11:41:08,Free,0,0,Calm,Unable to login with my account Unable to logi...,unable to login with my account unable to logi...
1,2,Unable to login with my account,Unable to login with my account. Thank you for...,email,Login,High,2026-02-11 04:41:08,Free,0,0,Appreciative,Unable to login with my account Unable to logi...,unable to login with my account unable to logi...
2,3,Password reset link not working,Password reset link not working. Great service...,web,Login,Medium,2026-01-21 14:41:08,Free,0,0,Appreciative,Password reset link not working Password reset...,password reset link not working password reset...
3,4,Account locked after multiple attempts,Account locked after multiple attempts. I have...,email,Login,High,2026-01-21 21:41:08,Free,0,0,Frustrated,Account locked after multiple attempts Account...,account locked after multiple attempts account...
4,5,Getting invalid credentials error,Getting invalid credentials error. This is com...,chat,Login,Medium,2026-02-10 22:41:08,Free,0,0,Angry,Getting invalid credentials error Getting inva...,getting invalid credentials error getting inva...


In [9]:
X = df[[
    "category",
    "priority",
    "customer_plan",
    "support_sentiment",
    "sla_breached"
]]

y = df["escalated"]


In [10]:
X = pd.get_dummies(X, drop_first=True)


In [11]:
X

,sla_breached,category_Billing,category_Bug,category_Feature Request,category_Login,priority_High,priority_Low,priority_Medium,customer_plan_Free,customer_plan_Pro,support_sentiment_Appreciative,support_sentiment_Calm,support_sentiment_Frustrated,support_sentiment_Urgent
0,0,False,False,False,True,True,False,False,True,False,False,True,False,False
1,0,False,False,False,True,True,False,False,True,False,True,False,False,False
2,0,False,False,False,True,False,False,True,True,False,True,False,False,False
3,0,False,False,False,True,True,False,False,True,False,False,False,True,False
4,0,False,False,False,True,False,False,True,True,False,False,False,False,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
995,0,False,False,False,False,False,True,False,True,False,False,False,False,True
996,0,False,False,False,False,False,False,True,False,True,False,False,True,False
997,0,False,False,False,False,False,False,True,False,True,False,True,False,False
998,0,False,False,False,False,False,False,True,False,False,False,False,False,True


In [12]:
from src.models.escalation_model import train_escalation_model

model = train_escalation_model(X, y)


Escalation Prediction Report:

              precision    recall  f1-score   support

           0       1.00      1.00      1.00       166
           1       1.00      1.00      1.00        34

    accuracy                           1.00       200
   macro avg       1.00      1.00      1.00       200
weighted avg       1.00      1.00      1.00       200

ROC-AUC Score: 1.0


In [14]:
import joblib
joblib.dump(model, "../models/escalation_model.pkl")

['../models/escalation_model.pkl']

In [15]:
import joblib

joblib.dump(X.columns.tolist(), "../models/escalation_feature_columns.pkl")


['../models/escalation_feature_columns.pkl']

## test run

In [17]:
from src.services.inference_service import predict_ticket

sample_ticket = {
    "subject": "Payment failed but amount deducted",
    "description": "This is completely unacceptable and blocking our operations.",
    "priority": "Critical",
    "customer_plan": "Enterprise",
    "sla_breached": 1
}

result = predict_ticket(sample_ticket)
print(result)


{'predicted_category': 'Billing', 'predicted_support_sentiment': 'Angry', 'escalation_probability': 0.995, 'escalation_flag': 1}
